In [ ]:


import pathlib
import sqlite3

import geopandas as gpd

from utils import fetch_handbook_dataframe

data_dir = pathlib.Path("..").resolve() / "data"
gpkg = str(data_dir / "ppm.gpkg")
# Create connection and load spatialite extension
df = gpd.read_file(gpkg, layer="ministers")
aph_df = fetch_handbook_dataframe(df)
aph_df

In [7]:
# Tidu up and create sub tables
ministers_table = aph_df[
    ["PHID", "GivenName", "MiddleNames", "FamilyName", "PreferredName", "DisplayName", "DateOfBirth", "PlaceOfBirth",
     "Image", "Gender", "MaritalStatus", "CountryOfBirth", "StateOfBirth", "Party", "PartyAbbrev", "SenateState",
     "State", "StateAbbrev", "InCurrentParliament", "ServiceHistory_Start", "ServiceHistory_End", "ServiceHistory_Days",
     "ElectedMemberNo", "ElectedSenatorNo", "FirstNations"]]
ministers_occupations = aph_df[["PHID", "Occupations"]].explode("Occupations")
ministers_secondary_occupations = aph_df[["PHID", "SecondaryOccupations"]].explode("SecondaryOccupations")
ministers_qualifications = aph_df[["PHID", "Qualifications"]].explode("Qualifications")

In [10]:
# Write data to gpkg
db_con = sqlite3.connect(gpkg)
ministers_table.to_sql("ministers_aph", db_con, if_exists="replace")
ministers_occupations.dropna().to_sql("ministers_occupations", db_con, if_exists="replace", index_label="fid")
ministers_secondary_occupations.dropna().to_sql("ministers_secondary_occupations", db_con, if_exists="replace",
                                                index_label="fid")
ministers_qualifications.dropna().to_sql("ministers_secondary_qualifications", db_con, if_exists="replace",
                                         index_label="fid")
aph_df.to_parquet(data_dir / "aph_all.parquet")

361

In [1]:
import pandas as pd

import pathlib
import geopandas as gpd

data_dir = pathlib.Path("..").resolve() / "data"
gpkg = str(data_dir / "ppm.gpkg")

aus_schools = gpd.read_file(data_dir / "school-location-2022.csv")
aus_schools["geometry"] = gpd.points_from_xy(aus_schools.Longitude, aus_schools.Latitude)
aus_schools.set_crs("EPSG:4326", inplace=True)
df = gpd.read_file(gpkg, layer="education")
df.set_crs("EPSG:4326", inplace=True)
aus_schools = aus_schools[aus_schools["School Type"].isin(["Secondary", "Combined"])]

In [34]:
merged = pd.merge(df, aus_schools, how="inner", left_on=df.school_name, right_on=aus_schools["School Name"])
matched_schools = merged[merged.geometry_x.buffer(0.01).intersects(merged.geometry_y)]
cleaned_matched_schools = matched_schools[
    ["school_name", "school_link", "operational_status", "ACARA SML ID", "Suburb", "State", "Postcode", "School Sector",
     "School Type", "ABS Remoteness Area", "ABS Remoteness Area Name", "Meshblock", "Statistical Area 1",
     "Statistical Area 2", "Statistical Area 2 Name", "Statistical Area 3", "Statistical Area 3 Name",
     "Statistical Area 4", "Statistical Area 4 Name", "Local Government Area", "Local Government Area Name",
     "State Electoral Division", "State Electoral Division Name", "Commonwealth Electoral Division",
     "Commonwealth Electoral Division Name", "geometry_x"]]
cleaned_matched_schools.set_geometry("geometry_x", inplace=True)
cleaned_matched_schools.rename_geometry("geometry", inplace=True)
cleaned_matched_schools.to_file(str(data_dir / "ppm.gpkg"), layer='secondary_schools', driver="GPKG", )

/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.rename(columns={geometry_col: col}, inplace=inplace)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.p

In [2]:
aus_schools_reproj = aus_schools.to_crs("EPSG:9473")
df_repoj = df[df.is_high_school > 0].to_crs("EPSG:9473")
matched_dist = df_repoj.sjoin_nearest(aus_schools_reproj, how="left", max_distance=1000)
merged = pd.merge(df, aus_schools, how="inner", left_on=df.school_name, right_on=aus_schools["School Name"])
merged.set_crs("EPSG:4326").to_crs("EPSG:9473")

AttributeError: You are calling a geospatial method on the GeoDataFrame, but the active geometry column ('geometry') is not present. 
There are columns with geometry data type (['geometry_x', 'geometry_y']), and you can either set one as the active geometry with df.set_geometry("name") or access the column as a GeoSeries (df["name"]) and call the method directly on it.

In [26]:
matched_dist

,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,geometry,...,Statistical Area 3,Statistical Area 3 Name,Statistical Area 4,Statistical Area 4 Name,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name
1,2.0,"Mazenod College, Perth",http://www.wikidata.org/entity/Q1914698,0,1,0,NaN,Point(116.039 -32.0067),NaN,POINT (-1490830.304 -3581328.902),...,50605,Kalamunda,506,Perth - South East,54200,Kalamunda,52702,Kalamunda (East Metropolitan),509,Hasluck
3,4.0,Anglican Church Grammar School,http://www.wikidata.org/entity/Q4763439,0,1,0,NaN,Point(153.054 -27.4819),NaN,POINT (2045962.495 -3149848.066),...,30502,Brisbane Inner - East,305,Brisbane Inner City,31000,Brisbane,30076,South Brisbane,313,Griffith
4,5.0,Aquinas College,http://www.wikidata.org/entity/Q4782971,0,1,0,NaN,Point(145.235555555 -37.823055555),NaN,POINT (1170372.722 -4195626.036),...,21103,Maroondah,211,Melbourne - Outer East,24410,Maroondah,27101,Ringwood (Eastern Metropolitan),211,Deakin
5,6.0,Armadale Senior High School,http://www.wikidata.org/entity/Q4792525,0,1,0,NaN,Point(116.015 -32.1617),NaN,POINT (-1490896.410 -3598813.109),...,50601,Armadale,506,Perth - South East,50210,Armadale,50202,Armadale (East Metropolitan),502,Burt
5,6.0,Armadale Senior High School,http://www.wikidata.org/entity/Q4792525,0,1,0,NaN,Point(116.015 -32.1617),NaN,POINT (-1490896.410 -3598813.109),...,50601,Armadale,506,Perth - South East,50210,Armadale,50202,Armadale (East Metropolitan),502,Burt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,198.0,Victorian School of Correspondence,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(143.3906074 -36.9847807),NaN,POINT (1015944.306 -4087658.653),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,217.0,Westfields Sports High School,http://www.wikidata.org/entity/Q7988785,0,1,0,NaN,Point(150.9199898 -33.867973),NaN,POINT (1734669.468 -3824698.788),...,12702,Fairfield,127,Sydney - South West,12850,Fairfield,10029,Fairfield,128,McMahon
278,183.0,Wilson Park Secondary College,https://www.wikidata.org/wiki/Q7604726,0,1,0,amalgamated,Point(144.574444 -37.708333),NaN,POINT (1113350.803 -4177123.446),...,21304,Melton - Bacchus Marsh,213,Melbourne - West,24650,Melton,24808,Melton (Western Victoria),218,Gorton
279,186.0,Wynyard High School,http://www.wikidata.org/entity/Q916063,0,1,0,NaN,Point(145.7169952 -40.9868049),NaN,POINT (1175621.137 -4544906.794),...,60401,Burnie - Ulverstone,604,West and North West,65410,Waratah-Wynyard,60209,Braddon (Murchison),602,Braddon


In [3]:
from thefuzz import fuzz


def calc_dist(x):
    if isinstance(x["School Name"], str) and isinstance(x["school_name"], str):
        s1, s2 = x["school_name"], x["School Name"]
        return fuzz.ratio(s1, s2)
    return 0


matched_dist["school_name_str_match"] = matched_dist.apply(calc_dist, axis=1)
merged["school_name_str_match"] = merged.apply(calc_dist, axis=1)

In [4]:
all_merged = pd.concat([matched_dist, merged])

/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as GDA2020 / Australian Albers (the single non-null crs provided).
  warnings.warn(
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


In [5]:
all_merged_deduped =  all_merged.sort_values("school_name_str_match").drop_duplicates("school_name", keep="last")

In [13]:
all_merged_deduped[all_merged_deduped["ACARA SML ID"].isna()]

,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,geometry,...,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name,school_name_str_match,key_0,geometry_x,geometry_y
232,219.0,Harvey Agricultural Senior High School,NaN,0,1,0,NaN,Point(115.8772958 -33.1340183),NaN,POINT (-1489993.022 -3708241.967),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
230,201.0,Greythorn High School,http://www.wikidata.org/entity/Q5608510,0,1,0,NaN,Point(145.096 -37.8017),1.0,POINT (1158316.899 -4192010.732),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
203,NaN,Boronia Heights College,http://www.wikidata.org/entity/Q4946270,0,1,0,amalgamated,Point(145.300833 -37.86),1.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
138,141.0,St. Paul's School,http://www.wikidata.org/entity/Q7595174,0,1,0,NaN,Point(121.52749 25.02591),NaN,POINT (-1453385.237 2238181.201),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
267,231.0,St Andrew Muar Johor,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(102.5729627 2.05415),NaN,POINT (-3575232.595 -205975.521),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
133,136.0,Norwood High School,http://www.wikidata.org/entity/Q62516299,0,1,0,NaN,Point(144.961 -37.8014),NaN,POINT (17865737.514 -9060438.022),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
271,190.0,Swanbourne High School,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(115.76678 -31.9711404),NaN,POINT (-1516625.221 -3580586.069),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
78,81.0,Enfield High School,http://www.wikidata.org/entity/Q5377304,0,1,0,closed,Point(138.6044383 -34.849003347396675),NaN,POINT (601825.541 -3820876.028),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
276,198.0,Victorian School of Correspondence,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(143.3906074 -36.9847807),NaN,POINT (1015944.306 -4087658.653),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
237,244.0,Kambalda Senior High School,NaN,0,1,0,NaN,Point(121.662514 -31.201572),NaN,POINT (-974312.928 -3437397.597),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None


In [6]:
all_merged_deduped

,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,geometry,...,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name,school_name_str_match,key_0,geometry_x,geometry_y
276,198.0,Victorian School of Correspondence,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(143.3906074 -36.9847807),NaN,POINT (1015944.306 -4087658.653),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
78,81.0,Enfield High School,http://www.wikidata.org/entity/Q5377304,0,1,0,closed,Point(138.6044383 -34.849003347396675),NaN,POINT (601825.541 -3820876.028),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
267,231.0,St Andrew Muar Johor,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(102.5729627 2.05415),NaN,POINT (-3575232.595 -205975.521),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
230,201.0,Greythorn High School,http://www.wikidata.org/entity/Q5608510,0,1,0,closed,Point(145.096 -37.8017),1.0,POINT (1158316.899 -4192010.732),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
232,219.0,Harvey Agricultural Senior High School,NaN,0,1,0,NaN,Point(115.913733 -33.129635),NaN,POINT (-1489993.022 -3708241.967),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,NaN,Marist Regional College,http://www.wikidata.org/entity/Q6765687,0,1,0,NaN,Point(145.886293 -41.048046),NaN,POINT (1189351.115 -4553098.855),...,60610,Burnie,60209,Braddon (Murchison),602,Braddon,100,NaN,None,None
205,NaN,Clairvaux MacKillop College,http://www.wikidata.org/entity/Q5125395,0,1,0,NaN,Point(153.0783765 -27.5529882),0.0,POINT (2046999.966 -3158105.419),...,31000,Brisbane,30049,Mansfield,302,Bonner,100,NaN,None,None
204,NaN,Murwillumbah High School,http://www.wikidata.org/entity/Q6939835,0,1,0,NaN,Point(153.3905989 -28.3346911),0.0,POINT (2062424.449 -3249614.559),...,17550,Tweed,10045,Lismore,138,Richmond,100,NaN,None,None
214,237.0,Adelaide High School,http://www.wikidata.org/entity/Q4681726,0,1,0,NaN,Point(138.586967 -34.9255505),1.0,POINT (599795.898 -3829300.774),...,40070,Adelaide,40001,Adelaide,401,Adelaide,100,NaN,None,None


In [ ]:
all_merged_deduped.to_file(gpkg, layer='education__acara', driver="GPKG")